In [1]:
import os
import logging
from typing import Any, Set


from automata.config.base import AgentConfigName
from automata.singletons.dependency_factory import dependency_factory, DependencyFactory
from automata.singletons.py_module_loader import py_module_loader
from automata.symbol.graph import SymbolGraph
from automata.symbol_embedding.base import SymbolCodeEmbedding
from automata.symbol_embedding.vector_databases import (
    ChromaSymbolEmbeddingVectorDatabase,
)

    
logger = logging.getLogger(__name__)

In [2]:
py_module_loader.reset()
dependency_factory.reset()
project_name = "automata"
py_module_loader.initialize(f"..", project_name) # ! Can replace with your own project


In [3]:
symbol_graph = SymbolGraph(
    os.path.join(DependencyFactory.DEFAULT_SCIP_FPATH, f"{project_name}.scip")
)

code_embedding_db = ChromaSymbolEmbeddingVectorDatabase(
    project_name,
    persist_directory=DependencyFactory.DEFAULT_CODE_EMBEDDING_FPATH,
    factory=SymbolCodeEmbedding.from_args,
)

In [4]:
symbol_graph.is_synchronized = True
len(symbol_graph.get_sorted_supported_symbols()),len(code_embedding_db)

(2534, 727)

In [5]:
symbol_graph.get_sorted_supported_symbols()[-1]

Symbol(scip-python python automata 189bb1fb0a9a91262c1280413fa8542b94ebdca6 `automata.tools.factory`/logger., scip-python, Package(python automata 189bb1fb0a9a91262c1280413fa8542b94ebdca6), (Descriptor(automata.tools.factory, 1), Descriptor(logger, 3)))

In [6]:
code_embedding_db.get_ordered_embeddings()[-1].symbol

Symbol(scip-python python automata 189bb1fb0a9a91262c1280413fa8542b94ebdca6 `automata.tools.factory`/AgentToolFactory#create_tools_from_builder()., scip-python, Package(python automata 189bb1fb0a9a91262c1280413fa8542b94ebdca6), (Descriptor(automata.tools.factory, 1), Descriptor(AgentToolFactory, 2), Descriptor(create_tools_from_builder, 4)))

In [7]:
dependency_factory.set_overrides(
    **{
        "symbol_graph": symbol_graph,
        "code_embedding_db": code_embedding_db,
    }
)


symbol_search = dependency_factory.get('symbol_search')
symbol_code_embedding_handler = dependency_factory.get('symbol_code_embedding_handler')


In [8]:
raw_ranks = symbol_search.symbol_rank.get_ordered_ranks()
print(f"Demonstrating SymbolRank (on {project_name}) \n")
for i,rank in enumerate(raw_ranks[0:10]):
    print(f"rank {i} = {rank[0].full_dotpath} with rank {rank[1]:.3f}")

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 692/692 [00:01<00:00, 396.22it/s]

Demonstrating SymbolRank (on automata) 

rank 0 = automata.singletons.github_client.GitHubClient with rank 0.007
rank 1 = automata.core.base.database.vector.VectorDatabaseProvider with rank 0.006
rank 2 = automata.core.base.database.vector.ChromaVectorDatabase with rank 0.005
rank 3 = automata.symbol.base.Symbol with rank 0.004
rank 4 = automata.singletons.dependency_factory.DependencyFactory with rank 0.004
rank 5 = automata.core.base.database.vector.JSONVectorDatabase with rank 0.004
rank 6 = automata.llm.foundation.LLMConversation with rank 0.004
rank 7 = automata.symbol.graph.symbol_graph.SymbolGraph with rank 0.004
rank 8 = automata.config.base.AgentConfigBuilder with rank 0.004
rank 9 = automata.symbol_embedding.vector_databases.ChromaSymbolEmbeddingVectorDatabase with rank 0.004


In [9]:
query = "How do I create an agent?"

embedding_similarity_calculator = dependency_factory.get('embedding_similarity_calculator')
code_embeddings = symbol_code_embedding_handler.get_ordered_embeddings()

print(f"Demonstrating Similarity Search (on {project_name}) for query = {query} \n")
code_similarity_results = embedding_similarity_calculator.calculate_query_similarity_dict(code_embeddings, query)
for i,rank in enumerate(list(code_similarity_results.items())[0:10]):
    print(f"rank {i} = {rank[0].full_dotpath} with distance {rank[1]:.3f}")

Demonstrating Similarity Search (on automata) for query = How do I create an agent? 

rank 0 = automata.agent.agent.Agent with distance 0.776
rank 1 = automata.agent.agent.AgentInstance with distance 0.773
rank 2 = automata.agent.agent.AgentInstance.create with distance 0.773
rank 3 = automata.agent.providers.OpenAIAutomataAgent with distance 0.767
rank 4 = automata.config.base.AgentConfigBuilder.build with distance 0.764
rank 5 = automata.config.openai_agent.OpenAIAutomataAgentConfig.get_llm_provider with distance 0.761
rank 6 = automata.cli.commands.run_agent with distance 0.760
rank 7 = automata.tools.factory.AgentToolFactory with distance 0.760
rank 8 = automata.tasks.executor.IAutomataTaskExecution._build_agent with distance 0.759
rank 9 = automata.tools.factory.AgentToolFactory.create_tools_from_builder with distance 0.755


In [10]:
symbol_rank_search_results = symbol_search.get_symbol_rank_results(query)
print(f"Demonstrating SymbolRank (on {project_name}) for query = {query} \n")
for i,rank in enumerate(symbol_rank_search_results[0:10]):
    print(f"rank {i} = {rank[0].full_dotpath} with rank {rank[1]:.3f}")

Demonstrating SymbolRank (on automata) for query = How do I create an agent? 

rank 0 = automata.config.base.AgentConfigBuilder with rank 0.010
rank 1 = automata.agent.providers.OpenAIAutomataAgent with rank 0.009
rank 2 = automata.agent.agent.Agent with rank 0.008
rank 3 = automata.agent.agent.AgentInstance with rank 0.008
rank 4 = automata.singletons.github_client.GitHubClient with rank 0.007
rank 5 = automata.config.openai_agent.OpenAIAutomataAgentConfigBuilder with rank 0.007
rank 6 = automata.singletons.dependency_factory.DependencyFactory with rank 0.007
rank 7 = automata.config.openai_agent.OpenAIAutomataAgentConfig with rank 0.007
rank 8 = automata.config.base.AgentConfig with rank 0.007
rank 9 = automata.tools.factory.AgentToolFactory with rank 0.006
